# Weather Station Data Scraping
#### NCDC API
Andrew McDonald // CSE 847 // April 2021

In [20]:
import requests
import urllib
import json
from datetime import datetime

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree

In [2]:
ignitions = gpd.read_file("2016_wildfires/usa_2016_ignitions.shp")
ignitions

,fire_ID,latitude,longitude,size,perimeter,start_date,start_DOY,end_date,end_DOY,duration,...,direction_,landcover,landcover_,tile_ID,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,1,20.8438,-156.4130,0.86,4.63,2016-10-26,300,2016-11-02,307,8,...,east,12,Croplands,h03v06,326625791,North America,United States of America,USA,18560000.0,POINT (-156.41374 20.84379)
1,2,20.8396,-156.4180,1.50,6.48,2016-10-28,302,2016-11-02,307,6,...,southeast,12,Croplands,h03v06,326625791,North America,United States of America,USA,18560000.0,POINT (-156.41833 20.83962)
2,3,20.8105,-156.4370,0.43,2.78,2016-11-29,334,2016-11-29,334,1,...,none,12,Croplands,h03v06,326625791,North America,United States of America,USA,18560000.0,POINT (-156.43708 20.81045)
3,4,20.6271,-156.2470,18.65,30.56,2016-02-16,47,2016-02-20,51,5,...,northwest,7,Open shrublands,h03v06,326625791,North America,United States of America,USA,18560000.0,POINT (-156.24786 20.62712)
4,5,19.8063,-155.8950,7.72,12.96,2016-03-20,80,2016-03-25,85,6,...,southeast,7,Open shrublands,h03v07,326625791,North America,United States of America,USA,18560000.0,POINT (-155.89512 19.80629)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11519,56791,40.0271,-74.4895,0.86,4.63,2016-06-26,178,2016-07-04,186,9,...,west,4,Deciduous Broadleaf forest,h12v04,326625791,North America,United States of America,USA,18560000.0,POINT (-74.48954 40.02711)
11520,56792,40.0063,-74.4994,3.00,9.26,2016-06-13,165,2016-06-19,171,7,...,west,5,Mixed forest,h12v04,326625791,North America,United States of America,USA,18560000.0,POINT (-74.49944 40.00628)
11521,56793,39.9979,-74.5012,3.86,12.04,2016-06-14,166,2016-06-21,173,8,...,south,5,Mixed forest,h12v05,326625791,North America,United States of America,USA,18560000.0,POINT (-74.50123 39.99794)
11522,97709,69.3521,-141.1620,0.21,1.85,2016-08-17,230,2016-08-17,230,1,...,none,7,Open shrublands,h13v02,326625791,North America,United States of America,USA,18560000.0,POINT (-141.16203 69.35210)


## 1. Obtain All Station Locations

In [11]:
api_token = "syKpWfFaNGhSsksXrJhcrwQspIYihUqW"
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/"
headers = {"token": api_token}

resource = "stations?"
query = {
    "datasetid": "GSOM",
    "startdate": "2016-01-01",
    "enddate": "2016-01-01",
    "limit": "1000"
}
query_str = urllib.parse.urlencode(query, doseq=False)
endpoint = base_url + resource + query_str
response = requests.get(endpoint, headers=headers)
results = json.loads(response.text)
results["metadata"]["resultset"]

{'offset': 1, 'count': 35678, 'limit': 1000}

In [18]:
stations = []
count = results["metadata"]["resultset"]["count"]
offset = 0
while offset < count:
    query["offset"] = offset
    query_str = urllib.parse.urlencode(query, doseq=False)
    endpoint = base_url + resource + query_str
    response = requests.get(endpoint, headers=headers)
    results = json.loads(response.text)
    stations.extend(results["results"])
    print(f"Finished page {offset//1000}")
    offset += 1000

Finished page 0
Finished page 1
Finished page 2
Finished page 3
Finished page 4
Finished page 5
Finished page 6
Finished page 7
Finished page 8
Finished page 9
Finished page 10
Finished page 11
Finished page 12
Finished page 13
Finished page 14
Finished page 15
Finished page 16
Finished page 17
Finished page 18
Finished page 19
Finished page 20
Finished page 21
Finished page 22
Finished page 23
Finished page 24
Finished page 25
Finished page 26
Finished page 27
Finished page 28
Finished page 29
Finished page 30
Finished page 31
Finished page 32
Finished page 33
Finished page 34
Finished page 35


In [25]:
stations_df = pd.DataFrame(stations)
stations_df

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
0,34.0,1944-04-01,2021-03-01,25.33300,"SHARJAH INTER. AIRP, AE",0.4643,GHCND:AE000041196,METERS,55.517
1,10.4,1983-01-01,2021-03-01,25.25500,"DUBAI INTERNATIONAL, AE",0.6362,GHCND:AEM00041194,METERS,55.364
2,26.8,1983-06-01,2021-03-01,24.43300,"ABU DHABI INTERNATIONAL, AE",0.6013,GHCND:AEM00041217,METERS,54.651
3,264.9,1994-04-01,2021-03-01,24.26200,"AL AIN INTERNATIONAL, AE",0.5123,GHCND:AEM00041218,METERS,55.609
4,977.2,1979-07-01,2020-12-01,34.21000,"HERAT, AF",0.0080,GHCND:AFM00040938,METERS,62.228
...,...,...,...,...,...,...,...,...,...
35674,7.0,1949-10-01,2020-03-01,19.28333,"WAKE ISLAND, US",0.7742,GHCND:WQW00041606,METERS,166.650
35675,641.0,1910-02-01,2017-11-01,-26.53300,"MANZINI MATSAPA AIR, WZ",0.3988,GHCND:WZ004455110,METERS,31.300
35676,1480.0,1956-07-01,2017-10-01,-17.91700,"HARARE KUTSAGA, ZI",0.6114,GHCND:ZI000067775,METERS,31.133
35677,1095.0,1951-07-01,2017-07-01,-20.06700,"MASVINGO, ZI",0.7541,GHCND:ZI000067975,METERS,30.867


## 2. Use KD Tree to Find Nearest Station

In [26]:
tree = KDTree(stations_df[["latitude", "longitude"]])
dist, idx = tree.query(ignitions[["latitude", "longitude"]], k=1)
idx

array([[32281],
       [32281],
       [32226],
       ...,
       [20541],
       [ 6073],
       [ 6073]], dtype=int64)

In [27]:
ignitions["nearest_station_idx"] = idx
ignitions

,fire_ID,latitude,longitude,size,perimeter,start_date,start_DOY,end_date,end_DOY,duration,...,landcover,landcover_,tile_ID,pop_est,continent,name,iso_a3,gdp_md_est,geometry,nearest_station_idx
0,1,20.8438,-156.4130,0.86,4.63,2016-10-26,300,2016-11-02,307,8,...,12,Croplands,h03v06,326625791,North America,United States of America,USA,18560000.0,POINT (-156.41374 20.84379),32281
1,2,20.8396,-156.4180,1.50,6.48,2016-10-28,302,2016-11-02,307,6,...,12,Croplands,h03v06,326625791,North America,United States of America,USA,18560000.0,POINT (-156.41833 20.83962),32281
2,3,20.8105,-156.4370,0.43,2.78,2016-11-29,334,2016-11-29,334,1,...,12,Croplands,h03v06,326625791,North America,United States of America,USA,18560000.0,POINT (-156.43708 20.81045),32226
3,4,20.6271,-156.2470,18.65,30.56,2016-02-16,47,2016-02-20,51,5,...,7,Open shrublands,h03v06,326625791,North America,United States of America,USA,18560000.0,POINT (-156.24786 20.62712),32923
4,5,19.8063,-155.8950,7.72,12.96,2016-03-20,80,2016-03-25,85,6,...,7,Open shrublands,h03v07,326625791,North America,United States of America,USA,18560000.0,POINT (-155.89512 19.80629),32941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11519,56791,40.0271,-74.4895,0.86,4.63,2016-06-26,178,2016-07-04,186,9,...,4,Deciduous Broadleaf forest,h12v04,326625791,North America,United States of America,USA,18560000.0,POINT (-74.48954 40.02711),20541
11520,56792,40.0063,-74.4994,3.00,9.26,2016-06-13,165,2016-06-19,171,7,...,5,Mixed forest,h12v04,326625791,North America,United States of America,USA,18560000.0,POINT (-74.49944 40.00628),20541
11521,56793,39.9979,-74.5012,3.86,12.04,2016-06-14,166,2016-06-21,173,8,...,5,Mixed forest,h12v05,326625791,North America,United States of America,USA,18560000.0,POINT (-74.50123 39.99794),20541
11522,97709,69.3521,-141.1620,0.21,1.85,2016-08-17,230,2016-08-17,230,1,...,7,Open shrublands,h13v02,326625791,North America,United States of America,USA,18560000.0,POINT (-141.16203 69.35210),6073


In [30]:
ignitions_stations = pd.merge(ignitions, stations_df,
                              left_on="nearest_station_idx",
                              right_index=True)
ignitions_stations["distance_from_station"] = np.sqrt(
    (ignitions_stations["latitude_x"] - ignitions_stations["latitude_y"])**2 +\
    (ignitions_stations["longitude_x"] - ignitions_stations["longitude_y"])**2
)
ignitions_stations

,fire_ID,latitude_x,longitude_x,size,perimeter,start_date,start_DOY,end_date,end_DOY,duration,...,elevation,mindate,maxdate,latitude_y,name_y,datacoverage,id,elevationUnit,longitude_y,distance_from_station
0,1,20.8438,-156.4130,0.86,4.63,2016-10-26,300,2016-11-02,307,8,...,70.1,1949-10-01,2016-10-01,20.85710,"PUUNENE 396, HI US",0.9875,GHCND:USC00518543,METERS,-156.43130,0.022623
1,2,20.8396,-156.4180,1.50,6.48,2016-10-28,302,2016-11-02,307,6,...,70.1,1949-10-01,2016-10-01,20.85710,"PUUNENE 396, HI US",0.9875,GHCND:USC00518543,METERS,-156.43130,0.021980
2,3,20.8105,-156.4370,0.43,2.78,2016-11-29,334,2016-11-29,334,1,...,48.2,1905-01-01,2016-10-01,20.79120,"KIHEI 311, HI US",0.6364,GHCND:USC00514489,METERS,-156.44210,0.019962
3,4,20.6271,-156.2470,18.65,30.56,2016-02-16,47,2016-02-20,51,5,...,1228.3,2013-01-01,2017-03-01,20.68420,"KAUPO GAP HAWAII, HI US",1.0000,GHCND:USR0000HKAU,METERS,-156.15190,0.110925
4,5,19.8063,-155.8950,7.72,12.96,2016-03-20,80,2016-03-25,85,6,...,709.0,2003-05-01,2021-03-01,19.79500,"PUU WAAWAA HAWAII, HI US",0.9487,GHCND:USR0000HPUW,METERS,-155.84530,0.050968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11517,56789,40.8521,-78.1691,0.43,2.78,2016-04-14,105,2016-04-14,105,1,...,524.3,1998-04-01,2021-03-01,40.87291,"PHILIPSBURG 2 S, PA US",0.9927,GHCND:USC00366921,METERS,-78.21615,0.051447
11518,56790,40.4188,-76.7056,0.64,3.70,2015-11-14,318,2015-11-14,318,1,...,163.7,1998-05-01,2021-03-01,40.46093,"DEHART DAM, PA US",0.8872,GHCND:USC00362071,METERS,-76.74870,0.060271
11519,56791,40.0271,-74.4895,0.86,4.63,2016-06-26,178,2016-07-04,186,9,...,37.5,2013-08-01,2020-06-01,40.06628,"JACKSON TWP 2.2 S, NJ US",0.5904,GHCND:US1NJOC0053,METERS,-74.35698,0.138191
11520,56792,40.0063,-74.4994,3.00,9.26,2016-06-13,165,2016-06-19,171,7,...,37.5,2013-08-01,2020-06-01,40.06628,"JACKSON TWP 2.2 S, NJ US",0.5904,GHCND:US1NJOC0053,METERS,-74.35698,0.154536


In [31]:
ignitions_stations.to_file("2016_wildfires/usa_2016_ignitions_stations.shp")


<ipython-input-31-796e7e0ce02c>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ignitions_stations.to_file("2016_wildfires/usa_2016_ignitions_stations.shp")
